<a href="https://colab.research.google.com/github/lmcanavals/data_mining/blob/main/notebooks/ja_condensed_representations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing fim like a pro [check here](https://borgelt.net/pyfim.html)

In [3]:
%%script bash
curl -O https://borgelt.net/bin/py3/fim.so
sudo ln -sf "$(pwd)/fim.so" /usr/local/lib/python3.7/dist-packages/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  756k  100  756k    0     0   262k      0  0:00:02  0:00:02 --:--:--  262k


In [4]:
from fim import *
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set()
import time
import pandas as pd

In [9]:
url="https://raw.githubusercontent.com/lmcanavals/data_mining/main/data/"
url+="credit-german.txt"
url

'https://raw.githubusercontent.com/lmcanavals/data_mining/main/data/credit-german.txt'

In [ ]:
help(eclat)

In [10]:
creditGerman = pd.read_csv(url, sep='\t')
creditGerman.head()

,checking_status,disc_duration,credit_history,purpose,disc_amount,savings_status,employment,personal_status,other_parties,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,lo_1_year,critical/other existing,radio/tv,1000_2000,no known savings,>=7,male single,none,real estate,>=55,none,own,two,skilled,one,yes,yes,good
1,0<=X<200,up_2_years,existing paid,radio/tv,up_2000,<100,1<=X<4,female div/dep/mar,none,real estate,<30,none,own,one,skilled,one,none,yes,bad
2,no checking,lo_1_year,critical/other existing,education,up_2000,<100,4<=X<7,male single,none,real estate,30<=X<55,none,own,one,unskilled resident,two,none,yes,good
3,<0,up_2_years,existing paid,furniture/equipment,up_2000,<100,4<=X<7,male single,guarantor,life insurance,30<=X<55,none,for free,one,skilled,two,none,yes,good
4,<0,1_2_years,delayed previously,new car,up_2000,<100,1<=X<4,male single,none,no known property,30<=X<55,none,for free,two,skilled,two,none,yes,bad


In [11]:
len(creditGerman)

1000

In [13]:
!pip install -U arules

In [15]:
import arules as ar
from arules.utils import five_quantile_based_bins, top_bottom_10, top_5_variant_variables

In [16]:
rules, supp_dict = ar.create_association_rules(creditGerman, max_cols=2,
                                               binning_method=five_quantile_based_bins)

Calculating all relevant supports
100%|██████████| 190/190 [00:00<00:00, 604.20it/s]

Calculating all feature level rules per variable level rule
100%|██████████| 342/342 [00:01<00:00, 329.44it/s]
Overall # of Rules:  3388


In [17]:
rules

,antecedent,consequent,rule print,ant_count,con_count,rule_count,ant_supp,con_supp,rule_supp,confidence,lift,# of all records
0,{'job': 'unemp/unskilled non res'},{'employment': 'unemployed'},job=unemp/unskilled non res ==> employment=une...,22,62,16,0.022,0.062,0.016,0.7273,11.7302,1000
1,{'employment': 'unemployed'},{'job': 'unemp/unskilled non res'},employment=unemployed ==> job=unemp/unskilled ...,62,22,16,0.062,0.022,0.016,0.2581,11.7302,1000
2,{'housing': 'for free'},{'property_magnitude': 'no known property'},housing=for free ==> property_magnitude=no kno...,108,154,104,0.108,0.154,0.104,0.9630,6.2530,1000
3,{'property_magnitude': 'no known property'},{'housing': 'for free'},property_magnitude=no known property ==> housi...,154,108,104,0.154,0.108,0.104,0.6753,6.2530,1000
4,{'credit_history': 'no credits/all paid'},{'purpose': 'business'},credit_history=no credits/all paid ==> purpose...,40,97,15,0.040,0.097,0.015,0.3750,3.8660,1000
...,...,...,...,...,...,...,...,...,...,...,...,...
3383,{'credit_history': 'existing paid'},{'existing_credits': 'two'},credit_history=existing paid ==> existing_cred...,530,333,46,0.530,0.333,0.046,0.0868,0.2606,1000
3384,{'housing': 'for free'},{'age': '<30'},housing=for free ==> age=<30,108,371,10,0.108,0.371,0.010,0.0926,0.2496,1000
3385,{'age': '<30'},{'housing': 'for free'},age=<30 ==> housing=for free,371,108,10,0.371,0.108,0.010,0.0270,0.2496,1000
3386,{'job': 'high qualif/self emp/mgm'},{'own_telephone': 'none'},job=high qualif/self emp/mgm ==> own_telephone...,148,596,21,0.148,0.596,0.021,0.1419,0.2381,1000


In [31]:
rules2 = rules[(rules['confidence'] >= 0.96) &
               (rules['rule_supp'] >= 0.6)]
rules2

,antecedent,consequent,rule print,ant_count,con_count,rule_count,ant_supp,con_supp,rule_supp,confidence,lift,# of all records
1676,{'other_parties': 'none'},{'foreign_worker': 'yes'},other_parties=none ==> foreign_worker=yes,907,963,880,0.907,0.963,0.880,0.9702,1.0075,1000
1682,{'job': 'skilled'},{'foreign_worker': 'yes'},job=skilled ==> foreign_worker=yes,630,963,611,0.630,0.963,0.611,0.9698,1.0071,1000
1692,{'num_dependents': 'one'},{'foreign_worker': 'yes'},num_dependents=one ==> foreign_worker=yes,845,963,819,0.845,0.963,0.819,0.9692,1.0065,1000
1804,{'existing_credits': 'one'},{'foreign_worker': 'yes'},existing_credits=one ==> foreign_worker=yes,633,963,609,0.633,0.963,0.609,0.9621,0.9991,1000
1822,{'other_payment_plans': 'none'},{'foreign_worker': 'yes'},other_payment_plans=none ==> foreign_worker=yes,814,963,782,0.814,0.963,0.782,0.9607,0.9976,1000
1823,{'housing': 'own'},{'foreign_worker': 'yes'},housing=own ==> foreign_worker=yes,713,963,685,0.713,0.963,0.685,0.9607,0.9976,1000


In [30]:
ar.present_rules_per_consequent(rules,
                                consequent={'foreign_worker':'yes'},
                                selection_function=top_5_variant_variables)

other_parties


,rule print,ant_supp,con_supp,rule_supp,confidence,lift
1676,other_parties=none ==> foreign_worker=yes,0.907,0.963,0.880,0.9702,1.0075
2398,other_parties=co applicant ==> foreign_worker=yes,0.041,0.963,0.037,0.9024,0.9371
2530,other_parties=guarantor ==> foreign_worker=yes,0.052,0.963,0.046,0.8846,0.9186


disc_duration


,rule print,ant_supp,con_supp,rule_supp,confidence,lift
1428,disc_duration=up_2_years ==> foreign_worker=yes,0.230,0.963,0.229,0.9957,1.0339
1586,disc_duration=1_2_years ==> foreign_worker=yes,0.411,0.963,0.403,0.9805,1.0182
2256,disc_duration=lo_1_year ==> foreign_worker=yes,0.359,0.963,0.331,0.9220,0.9574


purpose


,rule print,ant_supp,con_supp,rule_supp,confidence,lift
1381,purpose=domestic appliance ==> foreign_worker=yes,0.012,0.963,0.012,1.0000,1.0384
1382,purpose=education ==> foreign_worker=yes,0.050,0.963,0.050,1.0000,1.0384
1488,purpose=business ==> foreign_worker=yes,0.097,0.963,0.096,0.9897,1.0277
1556,purpose=radio/tv ==> foreign_worker=yes,0.280,0.963,0.275,0.9821,1.0199
1582,purpose=used car ==> foreign_worker=yes,0.103,0.963,0.101,0.9806,1.0183
1730,purpose=furniture/equipment ==> foreign_worker...,0.181,0.963,0.175,0.9669,1.0040
1904,purpose=repairs ==> foreign_worker=yes,0.022,0.963,0.021,0.9545,0.9912
2292,purpose=other ==> foreign_worker=yes,0.012,0.963,0.011,0.9167,0.9519
2330,purpose=new car ==> foreign_worker=yes,0.234,0.963,0.213,0.9103,0.9452


job


,rule print,ant_supp,con_supp,rule_supp,confidence,lift
1520,job=high qualif/self emp/mgm ==> foreign_worke...,0.148,0.963,0.146,0.9865,1.0244
1682,job=skilled ==> foreign_worker=yes,0.630,0.963,0.611,0.9698,1.0071
2198,job=unskilled resident ==> foreign_worker=yes,0.200,0.963,0.186,0.9300,0.9657
2342,job=unemp/unskilled non res ==> foreign_worker...,0.022,0.963,0.020,0.9091,0.9440


property_magnitude


,rule print,ant_supp,con_supp,rule_supp,confidence,lift
1468,property_magnitude=car ==> foreign_worker=yes,0.332,0.963,0.329,0.9910,1.0290
1514,property_magnitude=no known property ==> forei...,0.154,0.963,0.152,0.9870,1.0249
1988,property_magnitude=life insurance ==> foreign_...,0.232,0.963,0.220,0.9483,0.9847
2204,property_magnitude=real estate ==> foreign_wor...,0.282,0.963,0.262,0.9291,0.9648
